Funciona aqui pq eu preciso relembrar C, mas estudar como fazer um único código no C pra dar a saida direto

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
ano_i = 1900.00
ano_f = 1900.00

lat_i = -89.00
lat_f = -71.00

lon_i = -180.00
lon_f = -180.00

d_ano = 10.0
d_coord = 2.0

alt = 00.0

In [ ]:
# ok

lats = np.arange(lat_i, lat_f+d_coord, d_coord)
lats = lats.reshape(len(lats),1)
lons = np.arange(lon_i, lon_f+d_coord, d_coord)
lons = lons.reshape(len(lons),1)
anos = np.arange(ano_i, ano_f+d_ano, d_ano)
anos = anos.reshape(len(anos),1)

#coluna de longitudes ok
lons_2 = np.repeat(lons, len(lats), axis=1).reshape(len(lons)*len(lats), 1)
lons_3 = np.repeat(lons_2, len(anos), axis=1)
lons_3 = lons_3.T.reshape(lons_3.size, 1)

# coluna de latitudes ok
lats_2 = np.repeat(lats, len(lons), axis=1).T.reshape(len(lons)*len(lats), 1)
lats_3 = np.repeat(lats_2, len(anos), axis=1)
lats_3 = lats_3.T.reshape(lats_3.size, 1)

# coluna de anos ok
anos_2 = np.repeat(anos, len(lons_2), axis=1).reshape(len(lons)*len(lats)*len(anos), 1)

# strings extras para padrão de coordenadas (em graus) e altitude (em metros)
extras = [['D', 'M'+str(alt)]]*len(lons)*len(lats)*len(anos)

In [ ]:
# Padrão do input:     2019.5 D M0 70.3 30.8
M = np.concatenate((anos_2, extras, lats_3, lons_3), axis=1)
np.savetxt('my_input.txt', M, delimiter=" ", fmt="%s")

In [ ]:
# calcula o IGRF usando o Geomag 7.0
!geomag70.exe IGRF12.COF f my_input.txt my_output.txt

In [ ]:
# Lê o arquivo calculado
D_deg, D_min, I_deg, I_min = np.loadtxt('my_output.txt', skiprows=1, usecols=(5,6,7,8), dtype=str, unpack=True)
Date, Latitude, Longitude, F_nT, dD_min, dI_min, dF_nT = np.loadtxt('my_output.txt', skiprows=1, 
                                                                    usecols=(0, 3,4,13,14,15,20), dtype=float, unpack=True)

D_deg_num = np.array([float(i.split('d')[0]) for i in D_deg])
D_min_num = np.array([float(i.split('m')[0]) for i in D_min])
I_deg_num = np.array([float(i.split('d')[0]) for i in I_deg])
I_min_num = np.array([float(i.split('m')[0]) for i in I_min])


sinal = np.sign(D_deg_num)
sinal[sinal==0] = 1
D_graus = sinal * (np.abs(D_deg_num) + D_min_num/60.0)

sinal = np.sign(I_deg_num)
sinal[sinal==0] = 1
I_graus = sinal * (np.abs(I_deg_num) + I_min_num/60.0)

dD_graus = dD_min/60.0
dI_graus = dI_min/60.0

In [ ]:
# salva o arquivo em formato de tabela
IGRF = np.concatenate((Date.reshape(Date.size, 1), \
                       Latitude.reshape(Latitude.size, 1), \
                       Longitude.reshape(Longitude.size, 1), \
                       F_nT.reshape(F_nT.size, 1), \
                       D_graus.reshape(D_graus.size, 1), \
                       dD_graus.reshape(dD_graus.size, 1), \
                       I_graus.reshape(I_graus.size, 1), \
                       dI_graus.reshape(dI_graus.size, 1)), \
                      axis=1)

np.savetxt('IGRF.txt', IGRF)

In [ ]:
# apaga os outros arquivos
!del my_input.txt
!del my_output.txt